## Evaluation of the 2D model
Define parameters here

In [ ]:
# define path to config file
config_path = r"config_2D_test.yml"
# define GPU idx
GPU_idx = "1"
# define GPU max memory
GPU_max_memory = 22000

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_idx
import numpy as np
import yaml
import tensorflow as tf
from tensorflow.keras.layers import Input
from tifffile import imwrite
import matplotlib.pyplot as plt
from data_generator import data_generator_2D
from UNet_RCAN_2D import UNet_RCAN
from loss import custom_loss_with_l2_reg
from evaluation_parameters import nmse_psnr_ssim_2D

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=GPU_max_memory)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
# read config file
with open(config_path, "r") as f:
    config = yaml.safe_load(f)

data_config = config['data_test']
model_config = config['model']

x_test, y_test = data_generator_2D(data_config)

# display data
ix = np.random.randint(0,len(x_test),4)
fig = plt.figure(figsize=(15,7))

for i in range(4):
    fig.add_subplot(2,4, 2*i+1)
    cmap=plt.get_cmap('magma')
    plt.imshow(x_test[ix[i],:,:,0].squeeze(),cmap)
    plt.title('Low SNR',fontdict={'fontsize':18})
    plt_axis = plt.axis('off')

    fig.add_subplot(2,4, 2*i+2)
    cmap=plt.get_cmap('magma')
    plt.imshow(y_test[ix[i],:,:,0].squeeze(),cmap)
    plt.title('High SNR',fontdict={'fontsize':18})
    plt_axis = plt.axis('off')

### Generate predictions

In [ ]:
model_input = Input((data_config['patch_size'], data_config['patch_size'], 1))
model = UNet_RCAN(model_config)
model(np.zeros((1, data_config['patch_size'], data_config['patch_size'], 1)))
model.load_weights(model_config['save_dr'])

prediction1 = np.zeros(x_test.shape)
prediction2 = np.zeros(x_test.shape)

for i in range(len(x_test)):
    prediction1[i], prediction2[i] = model(x_test[i:i + 1], training=False)
    prediction2[i] = prediction2[i] / prediction2[i].max()
prediction2[prediction2 < 0] = 0

# display some predicted images
ix = np.random.randint(len(prediction2))
fig = plt.figure(figsize=(40,40))

plt.subplot(1, 3, 1)
plt.imshow(x_test[ix, :, :, 0] , cmap='magma')
plt.title('Low SNR Input',fontdict={'fontsize':40})
plt_axis = plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(prediction2[ix, :, :, 0] , cmap='magma')
plt.title('Prediction',fontdict={'fontsize':40})
plt_axis = plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(y_test[ix, :, :, 0] , cmap='magma')
plt.title('Ground Truth',fontdict={'fontsize':40})
plt_axis = plt.axis('off')

# compute scores
imageq_param = np.zeros((6,len(prediction2)))

imageq_param[0::2,:] = nmse_psnr_ssim_2D(x_test,y_test)
imageq_param[1::2,:] = nmse_psnr_ssim_2D(prediction2,y_test)

plt.rcParams.update({'font.size': 25})
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))
labels = ['noisy', 'prediction']

bplot1 = axes[0].boxplot([imageq_param[0,:],imageq_param[1,:]],
                         vert=True,  
                         patch_artist=True,  
                         labels=labels,showfliers=False)
axes[0].set_ylim(0, 1.2*imageq_param[0:2].max())
axes[0].set_title('NMSE',fontsize=30)

bplot2 = axes[1].boxplot([imageq_param[2,:],imageq_param[3,:]],
                         vert=True,  
                         patch_artist=True,  
                         labels=labels,showfliers=False) 
axes[1].set_ylim(0, 1.5*imageq_param[2:4].max())
axes[1].set_title('PSNR',fontsize=30)

bplot3 = axes[2].boxplot([imageq_param[4,:],imageq_param[5,:]],
                         vert=True,  
                         patch_artist=True,  
                         labels=labels,showfliers=False)

axes[2].set_ylim(0, 1)
cc= axes[2].set_title('MS-SSIM',fontsize=30)

np.savetxt(os.path.join(data_config['save_dr'], '', 'eval_param.csv'), np.transpose(imageq_param),
            header="NMSE_noisy,NMSE_prediction,PSNR_noisy,PSNR_prediction,SSIM_noisy,SSIM_prediction", delimiter=",")
plt.savefig(os.path.join(data_config['save_dr'], '', 'eval_param.tif'))
plt.show()
plt.close()

### Save results

In [ ]:
# store predicted images along with ground truth and noisy data
pred = (prediction2 * (2 ** 16 - 1)).astype(np.uint16)
noisy = (x_test * (2 ** 16 - 1)).astype(np.uint16)
gt = (y_test * (2 ** 16 - 1)).astype(np.uint16)

imwrite(os.path.join(data_config['save_dr'], '', 'pred.tif'), pred.squeeze(), imagej=True, metadata={'axes': 'TYX'})
imwrite(os.path.join(data_config['save_dr'], '', 'noisy.tif'), noisy.squeeze(), imagej=True, metadata={'axes': 'TYX'})
imwrite(os.path.join(data_config['save_dr'], '', 'gt.tif'), gt.squeeze(), imagej=True, metadata={'axes': 'TYX'})